<a href="https://colab.research.google.com/github/alekz7/MachineLearning/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

data = pd.read_csv('./titanic_data/train.csv')
print(data.shape)
# (891, 12)

(891, 12)


In [0]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [0]:
# La columna edad solo tiene 714 valores asi que llenaremos los faltantes con la media
# actualizacion para llenarlo con la media de los datos combinados
print(data['Age'].median()) # 28
#data['Age'] = data['Age'].fillna(data['Age'].median())

28.0


In [0]:
#TODO graficas con las relaciones de los datos para analizar y descartar
# sexo femenino estaba mas relacionado a sobrevivir
# edad es una caracteristica fuerte, 18 40 parece la edad ideal para sobrevivir
# fare tambien es importante, boletos de alto precio tenian mas oportunidad de sobrevivir
# Embarcacion tambien importa y es una caracteristica fuerte

In [0]:
# vamos a combinar el train y el test para hacer Feature Engineering
def combinar_datos():
  train = pd.read_csv('./titanic_data/train.csv')
  test  = pd.read_csv('./titanic_data/test.csv')
  backup_targets = train.Survived
  train.drop(['Survived'],1,inplace = True)
  train_mas_test = train.append(test)
  train_mas_test.reset_index(inplace = True)
  train_mas_test.drop(['index','PassengerId'], inplace = True, axis = 1)
  return train_mas_test
train_mas_test = combinar_datos()

In [0]:
train_mas_test.describe()

,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,29.881138,0.498854,0.385027,33.295479
std,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,80.000000,8.000000,9.000000,512.329200


In [0]:
print(train_mas_test['Age'].median())
# 28
data['Age'] = data['Age'].fillna(train_mas_test['Age'].median())

28.0


In [0]:
# la instruccion anterior llena los datos vacios de Age con el promedio de train mas test
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [0]:
# Creamos una columna mas con el titulo de la persona
titulos = set()
for name in data['Name']:
  titulos.add(name.split(',')[1].split('.')[0].strip())
print(titulos)
# {'the Countess', 'Don', 'Master', 'Rev', 'Mme', 'Jonkheer', 'Major', 'Sir', 'Ms', 'Mr', 'Mlle', 'Dr', 'Col', 'Mrs', 'Miss', 'Lady', 'Capt'}

{'the Countess', 'Don', 'Master', 'Rev', 'Mme', 'Jonkheer', 'Major', 'Sir', 'Ms', 'Mr', 'Mlle', 'Dr', 'Col', 'Mrs', 'Miss', 'Lady', 'Capt'}


In [0]:
# haciendo un diccionario con los equivalentes
Diccionario_titulos = {
    "Capt": "Oficial",
    "Col": "Oficial",
    "Major": "Oficial",
    "Jonkheer": "Realeza",
    "Don": "Realeza",
    "Sir" : "Realeza",
    "Dr": "Oficial",
    "Rev": "Oficial",
    "the Countess":"Realeza",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Realeza"
}

In [0]:
def obten_titulos():
  train_mas_test['Title'] = train_mas_test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
  train_mas_test['Title'] = train_mas_test.Title.map(Diccionario_titulos)
  return train_mas_test

In [0]:
train_mas_test = obten_titulos()
train_mas_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [0]:
# verificar donde el titulo quedo en null, solo un caso pero no esta en el set de entrenamiento
train_mas_test[train_mas_test['Title'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C,NaN


In [0]:
# tercera aproximacion a la edad, en lugar de llenarlo solo con el promedio de todo el data-set,
# se llenara de acuerdo a una agrupacion de otras columnas posterior a la agrupacion se aplicara el promedio
print(train_mas_test.iloc[:891].Age.isnull().sum())
#177
print(train_mas_test.iloc[891:].Age.isnull().sum())
#86


177
86
